In [43]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from scipy.stats import gamma, norm

from functools import reduce

from gproc.generative import sample_at_x
from gproc.laplace import chol_inverse

In [44]:
# Data sizes and dimension
D = 2
N = 50
M = 50

x_1 = np.random.uniform(-1, 1, N * D).reshape(-1, D) # Reshape to N x D matrix
x_2 = np.random.uniform(-1, 1, M * D).reshape(-1, D) # Reshape to N x D matrix

In [45]:
class BaseKernel:
    def __init__(self):
        pass

    def make_gram(self, x_1, x_2):
        raise NotImplementedError()
        
    def invert_gram(self):
        raise NotImplementedError()
    
    def constrain_params(self, unconstrained_param_array):
        raise NotImplementedError()

    def prior_log_pdf(self, param_array):
        raise NotImplementedError()

In [46]:
def squared_exponential(x_1, x_2, lengthscale=0.5, variance=1.0):
    """
    Also known as RBF.
    
    :param x_1, N x d matrix
    :param x_2, M x d matrix

    :param lengthscale, float
    :param variance, float

    :returns K, N x M matrix, K_{ij} = k(x_i, x_j; lengthscale, variance)
    """
    sq_diffs = cdist(x_1, x_2, metric = 'sqeuclidean')
    return variance * np.exp(-0.5 * sq_diffs / lengthscale)

squared_exponential(x_1, x_2).shape

(50, 50)

In [47]:
class SquaredExponential(BaseKernel):
    def __init__(self, lengthscale=0.5, variance=1.0):
        self.lengthscale = lengthscale
        self.variance = variance
        self.param_dim = 2
        super().__init__()
    
    def make_gram(self, x_1, x_2):
        self.gram = squared_exponential(x_1, x_2, self.lengthscale, self.variance)
        return self.gram 
    
    def invert_gram(self):
        self.inverse_gram = chol_inverse(self.gram)
        return self.inverse_gram
    
    def constrain_params(self, unconstrained_params):
        if unconstrained_params.shape[0] != self.param_dim:
            raise AssertionError('Parameter array not the same size as kernel parameter dimension')
        self.constrained_params = np.exp(unconstrained_params)
        return self.constrained_params
    
    def prior_log_pdf(self, d):
        self.prior = gamma.logpdf(self.constrained_params[0], a = 1, scale = np.sqrt(d))
        self.prior += gamma.logpdf(self.constrained_params[1], a = 1.2, scale = 1/0.2)
        return self.prior

kernel = SquaredExponential(lengthscale = 1, variance = 1)
kernel.make_gram(x_1, x_1)
kernel.constrain_params(np.zeros(2))
kernel.prior_log_pdf(1)

-3.0459514049176044

In [48]:
def rational_quadratic(x_1, x_2, lengthscale=0.5, variance=1.0, weighting=1.0):
    """
    Rational Quadratic Kernel, equivalent to adding together many Squared Exponential kernels with different 
    lengthscales. Weight parameter determine relative weighting of large and small scale variations. When
    the weighting goes to infinity, RQ = SE.
    
    :param x_1, N x d matrix
    :param x_2, M x d matrix

    :param lengthscale, float
    :param variance, float
    :param weighting, float

    :returns K, N x M matrix, K_{ij} = k(x_i, x_j; lengthscale, variance, weighting)
    """
    sq_diffs = cdist(x_1, x_2, metric = 'sqeuclidean')
    return variance * ( (1 + sq_diffs / 2*lengthscale * weighting) ** (-weighting) )

rational_quadratic(x_1, x_2).shape

(50, 50)

In [49]:
class RationalQuadratic(BaseKernel):
    def __init__(self, lengthscale=0.5, variance=1.0, weighting=1.0):
        self.lengthscale = lengthscale
        self.variance = variance
        self.weighting = weighting
        self.param_dim = 3
        super().__init__()
    
    def make_gram(self, x_1, x_2):
        self.gram = rational_quadratic(x_1, x_2, self.lengthscale, self.variance, self.weighting)
        return self.gram 
    
    def invert_gram(self):
        self.inverse_gram = chol_inverse(self.gram)
        return self.inverse_gram
    
    def constrain_params(self, unconstrained_params):
        if unconstrained_params.shape[0] != self.param_dim:
            raise AssertionError('Parameter array not the same size as kernel parameter dimension')
        self.constrained_params = np.exp(unconstrained_params)
        return self.constrained_params
    
    def prior_log_pdf(self, d):
        self.prior = gamma.logpdf(self.constrained_params[0], a = 1, scale = np.sqrt(d))
        self.prior += gamma.logpdf(self.constrained_params[1], a = 1.2, scale = 1/0.2)
        self.prior += gamma.logpdf(self.constrained_params[2], a = 0.00001, scale = 1/0.00001) # uninformative prior
        return self.prior

kernel = RationalQuadratic(lengthscale = 1, variance = 1, weighting = 1)
kernel.make_gram(x_1, x_1)
kernel.constrain_params(np.zeros(3))
kernel.prior_log_pdf(1)

-14.558996227068079

In [50]:
def periodic(x_1, x_2, lengthscale=0.5, variance=1.0, period=1.0):
    """
    The periodic kernel allows one to model functions which repeat themselves exactly.
    
    :param x_1, N x d matrix
    :param x_2, M x d matrix

    :param lengthscale, float
    :param variance, float
    :param period, float

    :returns K, N x M matrix, K_{ij} = k(x_i, x_j; lengthscale, variance, period)
    """
    diffs = cdist(x_1, x_2, metric = 'euclidean')
    return variance * np.exp(-2 * np.sin(np.pi * diffs / period)**2 / lengthscale)

periodic(x_1, x_2).shape

(50, 50)

In [51]:
class Periodic(BaseKernel):
    def __init__(self, lengthscale=0.5, variance=1.0, period=1.0):
        self.lengthscale = lengthscale
        self.variance = variance
        self.period = period
        self.param_dim = 3
        super().__init__()
    
    def make_gram(self, x_1, x_2):
        self.gram = periodic(x_1, x_2, self.lengthscale, self.variance, self.period)
        return self.gram 
    
    def invert_gram(self):
        self.inverse_gram = chol_inverse(self.gram)
        return self.inverse_gram
    
    def constrain_params(self, unconstrained_params):
        if unconstrained_params.shape[0] != self.param_dim:
            raise AssertionError('Parameter array not the same size as kernel parameter dimension')
        self.constrained_params = np.exp(unconstrained_params)
        return self.constrained_params
    
    def prior_log_pdf(self, d):
        self.prior = gamma.logpdf(self.constrained_params[0], a = 1, scale = np.sqrt(d))
        self.prior += gamma.logpdf(self.constrained_params[1], a = 1.2, scale = 1/0.2)
        self.prior += gamma.logpdf(self.constrained_params[2], a = 0.00001, scale = 1/0.00001) # uninformative prior
        return self.prior

kernel = Periodic(lengthscale = 1, variance = 1, period = 10)
kernel.make_gram(x_1, x_1)
kernel.constrain_params(np.zeros(3))
kernel.prior_log_pdf(1)

-14.558996227068079

In [52]:
def locally_periodic(x_1, x_2, lengthscale_sqe=0.5, variance=1.0, lengthscale_p =0.5, period=1.0):
    """
    A squared exponential kernel multiplied by a periodic kernel. Allows one to model periodic functions
    which can vary slowly over time.
    
    :param x_1, N x d matrix
    :param x_2, M x d matrix

    :param lengthscale, float
    :param variance, float
    :param period, float

    :returns K, N x M matrix, K_{ij} = k(x_i, x_j; lengthscale, variance, period)
    """
    
    diffs = cdist(x_1, x_2, metric = 'euclidean')
    sq_diffs = cdist(x_1, x_2, metric = 'sqeuclidean')
    
    K_period = np.exp(-2 * np.sin(np.pi * diffs / period)**2 / lengthscale_p)
    K_sqe = np.exp(-0.5 * sq_diffs / lengthscale_sqe)
    return variance * np.multiply(K_period, K_sqe)

locally_periodic(x_1, x_2, 1, 0.5, 0.5, 0.5)

array([[0.44058788, 0.01092957, 0.13111612, ..., 0.01430758, 0.00326764,
        0.00469625],
       [0.05628993, 0.01544622, 0.00197097, ..., 0.00205823, 0.09985992,
        0.06121271],
       [0.0041852 , 0.00844816, 0.00705453, ..., 0.30365039, 0.00778572,
        0.01022536],
       ...,
       [0.00281124, 0.06758655, 0.13390655, ..., 0.00262308, 0.41816697,
        0.01587509],
       [0.03599279, 0.04983472, 0.0169742 , ..., 0.25442687, 0.11076468,
        0.02124298],
       [0.40415466, 0.36215552, 0.03015017, ..., 0.076908  , 0.04661025,
        0.12075881]])

In [53]:
class LocallyPeriodic(BaseKernel):
    def __init__(self, lengthscale_sqe=0.5, variance=1.0, lengthscale_p =0.5, period=1.0):
        self.lengthscale_sqe = lengthscale_sqe
        self.variance = variance
        self.lengthscale_p = lengthscale_p
        self.period = period
        self.param_dim = 4
        super().__init__()
    
    def make_gram(self, x_1, x_2):
        self.gram = locally_periodic(x_1, x_2, self.lengthscale_sqe, self.variance, self.lengthscale_p, self.period)
        return self.gram 
    
    def invert_gram(self):
        self.inverse_gram = chol_inverse(self.gram)
        return self.inverse_gram
    
    def constrain_params(self, unconstrained_params):
        if unconstrained_params.shape[0] != self.param_dim:
            raise AssertionError('Parameter array not the same size as kernel parameter dimension')
        self.constrained_params = np.exp(unconstrained_params)
        return self.constrained_params
    
    def prior_log_pdf(self, d):
        self.prior = gamma.logpdf(self.constrained_params[0], a = 1, scale = np.sqrt(d))
        self.prior += gamma.logpdf(self.constrained_params[1], a = 1.2, scale = 1/0.2)
        self.prior += gamma.logpdf(self.constrained_params[2], a = 0.00001, scale = 1/0.00001) # uninformative prior
        self.prior += gamma.logpdf(self.constrained_params[3], a = 0.00001, scale = 1/0.00001) # uninformative prior
        return self.prior

kernel = LocallyPeriodic(lengthscale_sqe = 1, variance = 1, lengthscale_p = 0.5, period = 10)
kernel.make_gram(x_1, x_1)
kernel.constrain_params(np.zeros(4))
kernel.prior_log_pdf(1)

-26.072041049218555

In [54]:
def linear(x_1, x_2, constant_variance=0.5, variance=1.0, offset=1.0):
    """
    A linear kernel is a non-stationary kernel, which when used with a GP, is equivalent to
    Bayesian linear regression.
    
    :param x_1, N x d matrix
    :param x_2, M x d matrix

    :param constant_variance, float
    :param variance, float
    :param offset, float

    :returns K, N x M matrix, K_{ij} = k(x_i, x_j; constant_variance, variance, offset)
    """
    
    return constant_variance + variance*np.dot(x_1 - offset, x_2.T - offset)

linear(x_1, x_2)

array([[4.97739562, 5.13875461, 3.02925142, ..., 4.36382077, 1.43884431,
        1.98318415],
       [2.85665559, 2.40497004, 0.91674754, ..., 1.87625989, 1.70483555,
        1.19962963],
       [2.79226466, 2.7853959 , 1.64239385, ..., 2.36851027, 1.09915747,
        1.245824  ],
       ...,
       [1.77170676, 1.50588654, 0.68725745, ..., 1.21561043, 1.17939984,
        0.87420217],
       [3.83972814, 3.67497894, 1.90070337, ..., 3.03275252, 1.57788153,
        1.5632642 ],
       [5.53792717, 5.72975428, 3.36338987, ..., 4.86011723, 1.54278174,
        2.17041662]])

In [55]:
class Linear(BaseKernel):
    def __init__(self, constant_variance=0.5, variance=1.0, offset=1.0):
        self.constant_variance = constant_variance
        self.variance = variance
        self.offset = offset
        self.param_dim = 3
        super().__init__()
    
    def make_gram(self, x_1, x_2):
        self.gram = linear(x_1, x_2, self.constant_variance, self.variance, self.offset)
        return self.gram 
    
    def invert_gram(self):
        self.inverse_gram = chol_inverse(self.gram)
        return self.inverse_gram
    
    def constrain_params(self, unconstrained_params):
        if unconstrained_params.shape[0] != self.param_dim:
            raise AssertionError('Parameter array not the same size as kernel parameter dimension')
        self.constrained_params = np.concatenate((np.exp(unconstrained_params[0:2]), unconstrained_params[2].reshape(-1)))
        return self.constrained_params
    
    def prior_log_pdf(self, d):
        self.prior = gamma.logpdf(self.constrained_params[1], a = 1.2, scale = 1/0.2)
        self.prior += gamma.logpdf(self.constrained_params[1], a = 1.2, scale = 1/0.2)
        self.prior += norm.logpdf(self.constrained_params[2]) # Standard Gaussian prior for offset
        return self.prior

kernel = Linear(constant_variance=0.5, variance=1.0, offset=1.0)
kernel.make_gram(x_1, x_1)
kernel.constrain_params(np.zeros(3))
kernel.prior_log_pdf(1)

-5.010841343039882

In [56]:
class Additive(BaseKernel):
    def __init__(self, kernels):
        self.kernels = kernels
        self.param_dim = sum(k.param_dim for k in self.kernels)
        super().__init__()
    
    def make_gram(self, x_1, x_2):
        grams = [k.make_gram(x_1, x_2) for k in self.kernels]
        self.gram = sum(grams)
        return self.gram
        
    def invert_gram(self):
        self.inverse_gram = chol_inverse(self.gram)
        return self.inverse_gram

    def constrain_params(self, unconstrained_params):
        if unconstrained_params.shape[0] != self.param_dim:
            raise AssertionError('Parameter array not the same size as kernel parameter dimension')
            
        self.constrained_params = np.zeros(0)
        dim_count = 0
        for k in self.kernels:
            self.constrained_params = np.concatenate((self.constrained_params, k.constrain_params(unconstrained_params[dim_count:(dim_count + k.param_dim)])))
            dim_count += k.param_dim
        return self.constrained_params
    
    def prior_log_pdf(self, d):
        self.prior = 0
        for k in self.kernels:
            self.prior += k.prior_log_pdf(d)
        return self.prior

k1 = SquaredExponential(lengthscale = 1, variance = 1)
k2 = Periodic(lengthscale = 1, variance = 1, period = 1)
k3 = Linear(constant_variance = 0.5, variance = 1, offset = 1)

kadd = Additive([k1, k2, k3])

kadd.make_gram(x_1, x_1)
kadd.constrain_params(np.array([1, 2, 3, 4, 5, 6, 7, 8]))
kadd.prior_log_pdf(1)


In [65]:
class Multiplicative(BaseKernel):
    def __init__(self, kernels):
        self.kernels = kernels
        self.param_dim = sum(k.param_dim for k in self.kernels)
        super().__init__()
    
    def make_gram(self, x_1, x_2):
        grams = [k.make_gram(x_1, x_2) for k in self.kernels]
        self.gram = reduce(np.multiply, grams)
        return self.gram
        
    def invert_gram(self):
        self.inverse_gram = chol_inverse(self.gram)
        return self.inverse_gram
    
    def constrain_params(self, unconstrained_params):
        if unconstrained_params.shape[0] != self.param_dim:
            raise AssertionError('Parameter array not the same size as kernel parameter dimension')
            
        self.constrained_params = np.zeros(0)
        dim_count = 0
        for k in self.kernels:
            self.constrained_params = np.concatenate((self.constrained_params, k.constrain_params(unconstrained_params[dim_count:(dim_count + k.param_dim)])))
            dim_count += k.param_dim
        return self.constrained_params
    
    def prior_log_pdf(self, d):
        self.prior = 0
        for k in self.kernels:
            self.prior += k.prior_log_pdf(d)
        return self.prior
    
k1 = SquaredExponential(lengthscale = 1, variance = 1)
k2 = Periodic(lengthscale = 1, variance = 1, period = 10)
k3 = Linear(constant_variance = 0.5, variance = 1, offset = 1)

kmul = Multiplicative([k1, k2, k3])

kmul.make_gram(x_1, x_1)
kmul.constrain_params(np.array([1, 2, 3, 4, 5, 6, 7, 8]))

array([   2.71828183,    7.3890561 ,   20.08553692,   54.59815003,
        148.4131591 ,  403.42879349, 1096.63315843,    8.        ])

In [ ]:
def add(x_1, x_2, kernels):
    """
    Add together an arbitrary number of kernels.
    
    :param x_1, N x d matrix
    :param x_2, M x d matrix
    
    :param kernels, tuple, tuple consisting of kernel functions, and corresponding parameter dictionaries
    
    :returns K, N x M matrix, sum of kernel matrices 
    """
    
    return sum([
        kernel(x_1, x_2, **kernel_kwargs)
        for kernel, kernel_kwargs in kernels
    ])

kernels = (
    (locally_periodic, {'lengthscale':0.5, 'variance':1.0, 'period':1.0}),
    (periodic, {'lengthscale':0.5, 'variance':1.0, 'period':0.1}),
)

add(x_1, x_2, kernels).shape

(50, 25)

In [ ]:
def multiply(x_1, x_2, kernels):
    """
    Multiply together an arbitrary number of kernels.
    
    :param x_1, N x d matrix
    :param x_2, M x d matrix
    
    :param kernels, tuple, kernel functions and corresponding parameter dictionaries
    
    :returns K, N x M matrix, sum of kernel matrices 
    """
    
    grams = [
    kernel(x_1, x_2, **kernel_kwargs) 
    for kernel, kernel_kwargs in kernels
    ]
    
    return reduce(np.multiply, grams)

kernels = (
    (linear, {'constant_variance':0, 'variance':1.0, 'offset':0}),
    (periodic, {'lengthscale':0.5, 'variance':1.0, 'period':0.1}),
    (locally_periodic, {'lengthscale':0.5, 'variance':1.0, 'period':10})
    
)


multiply(x_1, x_2, kernels).shape

(50, 25)